In [119]:
!pip install XLsxWriter

In [120]:
import xlsxwriter
import re
import numpy as np
import pandas as pd

## Attribution Table Functions

In [121]:
def attribution_table(orig_df, transformed_df, model):
    model_coefs = model.coef_
    
    workbook = xlsxwriter.Workbook('test.xlsx')
    worksheet = workbook.add_worksheet()
    
    media_channels = transformed_df.columns.tolist()[:len(model_coefs)]
    
    write_parameters(media_channels, workbook, worksheet)
    write_revenue_change(media_channels, transformed_df, workbook, worksheet)
    write_spending_and_attribution(media_channels, orig_df, workbook, worksheet)
    
    workbook.close()

In [122]:
def write_parameters(media_channels, workbook, worksheet):
    row = 6
    col = 0
    
    bold = workbook.add_format({'bold': True})

    worksheet.write(row, 0, "Publisher-Tactic", bold)
    worksheet.write(row, 1, "Alpha", bold)
    worksheet.write(row, 2, "Lag", bold)
    worksheet.write(row, 3, "Theta", bold)
    worksheet.write(row, 4, "Power", bold)

    for feature in media_channels:
        row += 1

        f = re.findall("([\w|\s]+)_alpha=(\d.\d+)L=(\d+)theta=(\d+)", feature)[0] #f = orig_col, alpha, L, theta
        f = list(f)
        f.append(feature[-3:])
        f = [float(f[i]) if i!=0 else f[i] for i in range(len(f))]

        worksheet.write_string(row, col, f[0])
        col += 1

        for i in np.arange(1, 5):
            worksheet.write_number(row, col, f[i])
            col += 1
        col = 0

In [123]:
def write_revenue_change(media_channels, transformed_df, workbook, worksheet):
    row = label_row
    col = 5
    worksheet.write(row, col, "Channel Attributed Revenue", bold)
    row += 1
    
    for i in range(len(media_channels)):
        df_copy = transformed_df.copy()
        df_copy.loc[:, media_channels[i]] = 0
        pred_revenue_no_channel = []
        pred_revenue_with_channel = []
        for j in range(len(transformed_df)):
            no_channel_spending = np.array(df_copy.iloc[j,:len(media_channels)])
            all_channel_spending = np.array(transformed_df.iloc[j,:len(media_channels)])
            coefs = np.array(model_coefs)
            pred_revenue_no_channel.append(np.dot(no_channel_spending, coefs))
            pred_revenue_with_channel.append(np.dot(all_channel_spending, coefs))
        revenue_change = sum(pred_revenue_with_channel) - sum(pred_revenue_no_channel)
        worksheet.write_number(row, col, revenue_change)
        row += 1

In [124]:
def write_spending_and_attribution(media_channels, orig_df, workbook, worksheet):
    row = label_row
    col = 6
    worksheet.write(row, col, "Channel Spending", bold)
    row += 1
    
    for i in range(len(media_channels)):
        channel_spending = sum(orig_df.iloc[:,i])
        worksheet.write(row, col, channel_spending)
        row += 1
        
    row = label_row
    col += 1
    worksheet.write(row, col, "Attribution", bold)
    row += 1
    
    for i in range(len(media_channels)):
        worksheet.write_formula('H{}'.format(row + i + 1), '=F{}/G{}'.format(row + i + 1, row + i + 1))

## Test Attribution Functions

In [125]:
orig_df = pd.read_csv('original_dataset_proactiv.csv')
transformed_df = pd.read_csv('transformed_dataset_proactiv.csv')

In [126]:
model_coefs = [2.46538171e+01, 1.03919203e+02, 0.00000000e+00, 6.26124244e+01,
       1.68132077e+02, 5.88054997e+01, 3.97262842e+00, 0.00000000e+00,
       8.71137860e+03]

In [127]:
attribution_table(orig_df, transformed_df, model_coefs)

## Debugging Notebook

In [58]:
label_row = 6

https://xlsxwriter.readthedocs.io/tutorial01.html

In [60]:
workbook = xlsxwriter.Workbook('test.xlsx')
worksheet = workbook.add_worksheet()

inputs

In [61]:
media_channels = transformed_df.columns.tolist()[:len(model_coefs)]
media_channels

['Amazon_MediaCost_alpha=0.1L=2theta=0_power=0.6',
 'BingSearch_MediaCost_alpha=0.1L=2theta=0_power=0.1',
 'DV360_MediaCost_alpha=0.1L=2theta=0_power=0.6',
 'Facebook_MediaCost_alpha=0.9L=20theta=0_power=0.6',
 'GoogleSearch_MediaCost_alpha=0.6L=10theta=0_power=0.1',
 'Pinterest_MediaCost_alpha=0.1L=2theta=0_power=0.6',
 'Radio_MediaCost_alpha=0.1L=2theta=0_power=0.6',
 'Snapchat_MediaCost_alpha=0.1L=2theta=0_power=0.6',
 'YouTube_MediaCost_alpha=0.9L=6theta=0_power=0.1']

In [62]:
row = label_row
col = 0

bold = workbook.add_format({'bold': True})

worksheet.write(row, 0, "Publisher-Tactic", bold)
worksheet.write(row, 1, "Alpha", bold)
worksheet.write(row, 2, "Lag", bold)
worksheet.write(row, 3, "Theta", bold)
worksheet.write(row, 4, "Power", bold)

for feature in media_channels:
    row += 1
    
    f = re.findall("([\w|\s]+)_alpha=(\d.\d+)L=(\d+)theta=(\d+)", feature)[0] #f = orig_col, alpha, L, theta
    f = list(f)
    f.append(feature[-3:])
    f = [float(f[i]) if i!=0 else f[i] for i in range(len(f))]
    
    worksheet.write_string(row, col, f[0])
    col += 1
    
    for i in np.arange(1, 5):
        worksheet.write_number(row, col, f[i])
        col += 1
    col = 0


In [63]:
row = label_row
col = 5
worksheet.write(row, col, "Channel Attributed Revenue", bold)
row += 1

In [64]:
for i in range(len(media_channels)):
    df_copy = transformed_df.copy()
    df_copy.loc[:, media_channels[i]] = 0
    pred_revenue_no_channel = []
    pred_revenue_with_channel = []
    for j in range(len(transformed_df)):
        no_channel_spending = np.array(df_copy.iloc[j,:len(media_channels)])
        all_channel_spending = np.array(transformed_df.iloc[j,:len(media_channels)])
        coefs = np.array(model_coefs)
        pred_revenue_no_channel.append(np.dot(no_channel_spending, coefs))
        pred_revenue_with_channel.append(np.dot(all_channel_spending, coefs))
    revenue_change = sum(pred_revenue_with_channel) - sum(pred_revenue_no_channel)
    worksheet.write_number(row, col, revenue_change)
    row += 1    

In [65]:
row = label_row
col += 1
worksheet.write(row, col, "Channel Spending", bold)
row += 1

In [66]:
for i in range(len(media_channels)):
    channel_spending = sum(orig_df.iloc[:,i])
    worksheet.write(row, col, channel_spending)
    row += 1

In [67]:
row = label_row
col += 1
worksheet.write(row, col, "Attribution", bold)
row += 1

In [68]:
for i in range(len(media_channels)):
    worksheet.write_formula('H{}'.format(row + i + 1), '=F{}/G{}'.format(row + i + 1, row + i + 1))

In [69]:
workbook.close()